In [12]:
!python3 -m pip install clickhouse-driver --user

     |████████████████████████████████| 150 kB 2.5 MB/s eta 0:00:01


In [33]:
class MapWay:
    
    
    def __init__(self, 
                 nameAlgo=None, 
                 arrAsks=None, 
                 current_position=None, 
                 points=None, 
                 model=None, 
                 places=None):
        self.nameAlgo = nameAlgo
        self.arrAsks = arrAsks
        self.points = points
        self.model = model
        self.places = places
    
    def findBestWay(self, user=None, points=None):
        # 
        places_in_zone = []
        for place in self.places:
            if inZone(first_point=points[0], 
                      second_point=points[1], 
                      object_place = place):
                places_in_zone.append(place)
            
        '''
        Построение графа

        X
         \
          \
           \
            *-------

        '''
        
    def inZone(self, 
               first_point=None,
               second_point=None, 
               object_place=None):
        '''
            y                                         y
            ^                                         ^
            X----------------------                   --------------------Y
            |         *           |                   |   *               |   *
            |                     | *            *    |            *      |
            |    *         *      |                   |                   |
            |                     |                   |       *           |
        *   |                     |                   |                   |
            |         *       *   |    *              |                   | *
            |                     |               *   |   *               |
          * ----------------------Y > x               X-------------------- > x
          
        '''
        left_point = None
        right_point = None
        if first_point.longtitude <= second_point.longtitude:
            left_point = first_point
        else:
            left_point = second_point
        
        if left_point.latitude >= right_point.latitude:
            if (object_place.longtitude <= right_point.longtitude and 
                object_place.longtitude >= left_point.longtitude and 
                object_place.latitude <= left_point.latitude and 
                object_place.latitude >= right_point.latitude):
                return True
            else:
                return False
        else:
            if (object_place.longtitude <= right_point.longtitude and 
                object_place.longtitude >= left_point.longtitude and 
                object_place.latitude <= right_point.latitude and 
                object_place.latitude >= left_point.latitude):
                return True
            else:
                return False
            
class WorkWithDB:
    
    def __init__(self):
        pass
    
class User:
    
    def __init__(self, 
                 time=None, 
                 age=None, 
                 religion=None, 
                 sex=None, 
                 marital_status=None, 
                 other_info=None):
        '''
            Create or get dataset
            Сделал все метрики для лучшего понимания что собирать с людей
        '''
#         self.category = model.predict()
     
    def getCategory(self):
        return self.category

class Position:
    
    def __init__(self, 
                 latitude=None, 
                 longtitude=None, 
                 type_of_point=None):
        self.latitude = latitude
        self.longtitude = longtitude
        self.type_of_point = None

class Place:
    
    def __init__(self, 
                 name=None, 
                 position : Position = None, 
                 category=None):
        self.name = name
        self.position = position
        self.category = category
        self.length
